In [2]:
%matplotlib notebook
%matplotlib inline

VQA_PATH='/Data_Storage/Rui_Data_Space/multimodal/VQA'
GQA_PATH='/Data_Storage/Rui_Data_Space/multimodal/GQA'

import pandas as pd
import json
import pickle as pkl
import numpy as np
import h5py
import random
from PIL import Image
import IPython.display as display
from collections import defaultdict
import os

import matplotlib.pyplot as plt
import torch

def load_pkl(path):
    data=pkl.load(open(path,'rb'))
    return data

def load_json(path):
    data=json.load(open(path,'r'))
    return data

In [3]:
programs=load_json('meta_questions/questions/testdev_pred_programs.json')
print(len(programs))

12578


In [ ]:
programs=load_json('meta_questions/questions/trainval_unbiased_programs.json')
print(len(programs))

In [4]:
gqa_val_q=json.load(
    open(os.path.join(GQA_PATH,'original','testdev_balanced_questions.json')
         ,'r')
)
print (len(gqa_val_q))
names=list(gqa_val_q.keys())

12578


In [29]:
gqa_val_q=json.load(
    open(os.path.join(GQA_PATH,'original','val_balanced_questions.json')
         ,'r')
)
print (len(gqa_val_q))
names=list(gqa_val_q.keys())

132062


In [5]:
prog_names=[row[0] for row in programs]
print (len(prog_names))

12578


In [6]:
id_to_prog={}
for row in programs:
    id_to_prog[row[3]]=row[2]
print (len(id_to_prog))

12578


In [6]:
print (programs[0])

['n161313', 'Is it overcast ?', ['verify_f(overcast)'], '201307251', 'no']


In [14]:
invalid=0
for i,name in enumerate(names):
    if name not in id_to_prog:
        invalid+=1
    if i%10000==0:
        print ('Already finished',i,invalid)
print (invalid)

Already finished 0 0
Already finished 10000 0
Already finished 20000 0
Already finished 30000 0
Already finished 40000 0
Already finished 50000 0
Already finished 60000 0
Already finished 70000 0
Already finished 80000 0
Already finished 90000 0
Already finished 100000 0
Already finished 110000 0
Already finished 120000 0
Already finished 130000 0
0


In [8]:
import vqa_prepro.modify_program as clean_modules

def update_program(program):
    program=clean_modules.eliminate_obj_id(program)
    program=clean_modules.modify_choose(program)
    program=clean_modules.modify_diff_same(program)
    program=clean_modules.modify_verify(program)
    program=clean_modules.modify_relate(program)
    program=clean_modules.modify_filter(program)
    return program

In [9]:
def mine_filter(prog):
    new_prog=[]
    flag=False
    select_idx={i:0 for i in range(len(prog))}
    ops=[step['operation'] for step in prog]
    if 'attentionand' not in ops:
        return prog
    for k,p in enumerate(prog):
        module=p['operation']
        if  p['operation']=='select' and  prog[k+1]['operation']=='attentionand':
            flag=True
            for i in range(k,len(prog)):
                select_idx[i]+=1
            #print (select_idx)
            dep=prog[k+1]['dependencies'][0]
            dep-=select_idx[dep]
            new_prog.append(
                {'operation':'filter',
                 'argument':p['argument'],
                 'dependencies':[dep]}
            )
        elif p['operation']=='attentionand' and flag:
            flag=False
            continue
        else:
            if p['operation'] in ['and','or']:
                dep=[]
                exist_0=prog[p['dependencies'][0]]['dependencies'][0]
                exist_1=prog[p['dependencies'][1]]['dependencies'][0]
                dep.append(p['dependencies'][0]-select_idx[exist_0])
                dep.append(p['dependencies'][1]-select_idx[exist_1])
            else:
                dep=[dep-select_idx[k] for dep in p['dependencies']]
            new_prog.append(
                {'operation':p['operation'],
                 'argument':p['argument'],
                 'dependencies':dep}
            )
    return new_prog

def mine_clean_compare(prog):
    if prog[-1]['operation']=='compare':
        if len(prog)==4:
            new_prog=[]
            new_prog.append(prog[0])
            new_prog.append(prog[1])
            final=prog[-1]
            new_prog.append({
                'argument':final['argument'],
                'dependencies':[0,1],
                'operation':final['operation']
            })
            return new_prog
    return prog

def mine_choose(prog):
    if prog[-1]['operation']!='choose' or prog[-2]['operation']!='exist':
        return prog
    if 'to the left of' not in prog[-1]['argument'] and\
                'behind' not in prog[-1]['argument'] and\
                'above' not in prog[-1]['argument']:
        new_prog=[]
        choose_from=prog[-1]['argument']
        for k,p in enumerate(prog):
            if p['operation']=='select' and p['argument'][0] in choose_from:
                break
            else:
                new_prog.append(p)
        if len(new_prog)==0:
            new_prog.append(
                {'operation':'select',
                 'argument':choose_from[0],
                 'dependencies':[]}
            )
            new_prog.append(
                {'operation':'select',
                 'argument':choose_from[1],
                 'dependencies':[]}
            )
        new_prog.append(
            {'operation':'choose',
            'argument':choose_from,
            'dependencies':[len(new_prog)-1]}
        )
        
    else:
        ops=[step['operation'] for step in prog]
        #print (ops)
        select_idx=ops.index('exist')
        #print (select_idx)
        deps=prog[select_idx-1]['dependencies']
        dep_0=deps[0]
        dep_1=deps[1]
        choose_from=prog[-1]['argument']
        new_prog=[]
        new_prog.extend([prog[i] for i in range(dep_0)])
        new_prog.extend([prog[i] for i in range(dep_0+1,dep_1+1)])
        new_prog.append(
            {'operation':'choose',
             'argument':choose_from,
             'dependencies':[dep_0-1,len(new_prog)-1]}
        )
        #print (new_prog)
    if new_prog[0]['operation']=='select' and new_prog[1]['operation']=='select':
        if len(new_prog)==3 and len(new_prog[-1]['dependencies'])==1:
            new_prog[-1]['dependencies'].insert(0,0)
    return new_prog

def mine_not(prog):
    ops=[step['operation'] for step in prog]
    if 'attentionnot' not in ops:
        return prog
    select_idx={i:0 for i in range(len(prog))}
    new_prog=[]
    flag=True
    
    invalid=[]
    for k,step in enumerate(prog):
        if k in invalid:
            continue
        if step['operation']=='select' and prog[k+1]['operation']=='attentionnot':
            flag=False
            invalid.append(k+1)
            invalid.append(k+2)
            for i in range(k,len(prog)):
                select_idx[i]+=2
            new_prog.append(
                {
                    'operation':'filter',
                    'dependencies':[k-1-select_idx[k-1]],
                    'argument': ['not '+step['argument'][0]]
                })
        else:
            if flag:
                new_prog.append(step)
            else:
                dep=step['dependencies']
                new_dep=[p-select_idx[p] for p in dep]
                new_prog.append({
                    'operation':step['operation'],
                    'dependencies':new_dep,
                    'argument':step['argument']
                })
    return new_prog

def mine_clean_all(prog):
    choose_prog=mine_choose(prog)
    filter_prog=mine_filter(choose_prog)
    not_prog=mine_not(filter_prog)
    comp_prog=mine_clean_compare(not_prog)
    for step in comp_prog:
        if type(step['argument']) is not list:
            step['argument']=[step['argument']]
    return comp_prog

In [9]:
def from_meta_to_layout_query(meta):
    layout=[]
    for step in meta:
        info=step.split('(')
        op_info=info[0].split('_')
        op=op_info[0]
        arg=info[1][:-1].split(',')
        #print (op,arg)
        if op in ['select','filter']:
            arg=arg[-1]
            if op=='filter' and len(op_info)>1:
                if op_info[1]=='not':
                    arg='not '+arg
            args=[arg]
        elif op=='relate':
            op='relocate'
            if 'inv' in op_info:
                rela_symbol='s'
            else:
                rela_symbol='o'
            args=[arg[1],rela_symbol]
        elif op=='query':
            if len(op_info)==1:
                args=[arg[-1]]
            elif op_info[1]=='n':
                args=['name']
            elif op_info[1]=='f':
                args=[arg[-1]]
            elif op_info[1]=='h':
                args=['hposition']
        layout.append({
            'operation':op,
            'argument':args
        })
        if op=='relocate' and len(arg)==3:
            layout.append({
                'operation':'filter',
                'argument':[arg[-1]]
            })
        if op=='query' and len(op_info)>1 and op_info[1]=='f':
            layout.insert(0,{
                'operation':'select',
                'argument':['scene']
            })
    return layout

In [14]:
random.shuffle(names)
vis=0
for k,name in enumerate(names):
    if vis>10:
        break
    row=gqa_val_q[name]
    layout=row['semantic']
    clean_layout=update_program(layout)
    new_prog=mine_clean_all(clean_layout)
    #if len(new_prog)<6:
    #    continue
    if new_prog[-1]['operation']!='query':
        continue
    vis+=1
    
    print (row['question'])
    meta_layout=from_meta_to_layout_query(id_to_prog[name])
    print (id_to_prog[name])
    for k,step in enumerate(new_prog):
        meta_info=meta_layout[k]
        print('\t',step['operation'],step['argument'],
              '\t',meta_info['operation'],meta_info['argument']) 
        
    print (id_to_prog[name],'\n')

Which kind of furniture in the photo is not small?
['select(furniture)', 'filter_not([0],small)', 'query_n([1])']
	 select ['furniture'] 	 select ['furniture']
	 filter ['not small'] 	 filter ['not small']
	 query ['name'] 	 query ['name']
['select(furniture)', 'filter_not([0],small)', 'query_n([1])'] 

What is the length of the TV stand on the left?
['select(tv)', 'filter_h([0],left)', 'query([1],length)']
	 select ['tv stand'] 	 select ['tv']
	 filter ['left'] 	 filter ['left']
	 query ['length'] 	 query ['length']
['select(tv)', 'filter_h([0],left)', 'query([1],length)'] 

What is the drink made of?
['select(drink)', 'query([0],material)']
	 select ['drink'] 	 select ['drink']
	 query ['material'] 	 query ['material']
['select(drink)', 'query([0],material)'] 

Which kind of watercraft is behind the beach?
['select(beach)', 'relate_inv_name([0],behind,watercraft)', 'query_n([1])']
	 select ['beach'] 	 select ['beach']
	 relocate ['behind', 's'] 	 relocate ['behind', 's']
	 filter ['w

In [17]:
#random.shuffle(names)
vis=0
total={}
for k,name in enumerate(names):
    row=gqa_val_q[name]
    layout=row['semantic']
    if layout[-1]['operation']!='query':
        continue
    #clean_layout=update_program(layout)
    #new_prog=mine_clean_all(clean_layout)
    #if len(new_prog)<6:
    #    continue
    #if new_prog[-1]['operation']!='query':
    #    continue
    vis+=1
    
    meta_layout=from_meta_to_layout_query(id_to_prog[name])
    total[name]=meta_layout
print (len(total))

6805


In [18]:
pkl.dump(total,
         open(os.path.join('../mdetr/meta_generated_layout','dev_query.pkl'),'wb'))

In [38]:
random.shuffle(names)
vis=0
op_dict=defaultdict(int)
for k,name in enumerate(names):
    row=gqa_val_q[name]
    layout=row['semantic']
    clean_layout=update_program(layout)
    new_prog=mine_clean_all(clean_layout)
    if new_prog[-1]['operation']!='query':
        continue
    vis+=1
    
    meta=id_to_prog[name]
    for step in meta:
        info=step.split('(')
        op=info[0].split('_')[0]
        op_dict[op]+=1
        arg=info[1][:-1].split(',')
        
print(op_dict)

defaultdict(<class 'int'>, {'select': 65555, 'relate': 50188, 'query': 68167, 'filter': 14642})


In [44]:
random.shuffle(names)
vis=0
final_query=defaultdict(int)
op_dict=defaultdict(int)
for k,name in enumerate(names):
    row=gqa_val_q[name]
    layout=row['semantic']
    clean_layout=update_program(layout)
    new_prog=mine_clean_all(clean_layout)
    if new_prog[-1]['operation']!='query':
        continue
    vis+=1
    
    meta=id_to_prog[name]
    for step in meta:
        info=step.split('(')
        op=info[0].split('_')[0]
        if op=='query':
            final_query[info[0]]+=1
        op_dict[op]+=1
        arg=info[1][:-1].split(',')
        
print(op_dict)
print (final_query)

defaultdict(<class 'int'>, {'query': 68167, 'select': 65555, 'filter': 14642, 'relate': 50188})
defaultdict(<class 'int'>, {'query_f': 2612, 'query_n': 47463, 'query': 11590, 'query_h': 6502})


In [19]:
def from_meta_to_layout_verify(meta):
    layout=[]
    for step in meta:
        info=step.split('(')
        op_info=info[0].split('_')
        op=op_info[0]
        arg=info[1][:-1].split(',')
        #print (op,arg)
        if op in ['select','filter']:
            arg=arg[-1]
            if op=='filter' and len(op_info)>1:
                if op_info[1]=='not':
                    arg='not '+arg
            args=[arg]
        elif op=='relate':
            op='relocate'
            if 'inv' in op_info:
                rela_symbol='s'
            else:
                rela_symbol='o'
            args=[arg[1],rela_symbol]
        elif op=='exist':
            args=['?']
        elif op=='verify':
            if len(op_info)==1:
                layout.append({
                    'operation':'filter',
                    'argument':[arg[-1]]
                })
            elif op_info[1] in ['f','v','h']:
                if len(meta)==1:
                    layout.append({
                        'operation':'select',
                        'argument':['scene']
                    })
                layout.append({
                    'operation':'filter',
                    'argument':[arg[-1]]
                })
            elif 'rel' in op_info[1]:
                if 'inv' in op_info:
                    rela_symbol='s'
                else:
                    rela_symbol='o'
                layout.append({
                    'operation':'relocate',
                    'argument':[arg[1],rela_symbol]
                })
                layout.append({
                    'operation':'filter',
                    'argument':[arg[-1]]
                })
            op='exist'
            args=['?']
        layout.append({
            'operation':op,
            'argument':args
        })
        if op=='relocate' and len(arg)==3:
            layout.append({
                'operation':'filter',
                'argument':[arg[-1]]
            })
    return layout

In [35]:
random.shuffle(names)
vis=0
for k,name in enumerate(names):
    if vis>10:
        break
    row=gqa_val_q[name]
    layout=row['semantic']
    clean_layout=update_program(layout)
    new_prog=mine_clean_all(clean_layout)
    #if len(new_prog)<6:
    #    continue
    if row['types']['structural']!='verify':
        continue
    vis+=1
    
    print (row['question'])
    meta_layout=from_meta_to_layout_verify(id_to_prog[name])
    print (id_to_prog[name])
    for k,step in enumerate(new_prog):
        meta_info=meta_layout[k]
        print('\t',step['operation'],step['argument'],
              '\t',meta_info['operation'],meta_info['argument']
             ) 
        
    print (id_to_prog[name],'\n')

Does the cow's nose look black?
['select(cow)', 'relate_inv_name([0],of,nose)', 'verify([1],black)']
	 select ['cow'] 	 select ['cow']
	 relocate ['of', 's'] 	 relocate ['of', 's']
	 filter ['nose'] 	 filter ['nose']
	 filter ['black'] 	 filter ['black']
	 exist ['?'] 	 exist ['?']
['select(cow)', 'relate_inv_name([0],of,nose)', 'verify([1],black)'] 

Does the large horse look black?
['select(horse)', 'filter([0],large)', 'verify([1],black)']
	 select ['horse'] 	 select ['horse']
	 filter ['large'] 	 filter ['large']
	 filter ['black'] 	 filter ['black']
	 exist ['?'] 	 exist ['?']
['select(horse)', 'filter([0],large)', 'verify([1],black)'] 

Are there goggles in the image?
['select(goggles)', 'exist([0])']
	 select ['goggles'] 	 select ['goggles']
	 exist ['?'] 	 exist ['?']
['select(goggles)', 'exist([0])'] 

Are the drinks on the left side?
['select(drink)', 'verify_h([0],left)']
	 select ['drinks'] 	 select ['drink']
	 filter ['left'] 	 filter ['left']
	 exist ['?'] 	 exist ['?']
[

In [40]:
random.shuffle(names)
vis=0
total={}
for k,name in enumerate(names):
    row=gqa_val_q[name]
    layout=row['semantic']
    #clean_layout=update_program(layout)
    #new_prog=mine_clean_all(clean_layout)
    #if len(new_prog)<6:
    #    continue
    if row['types']['structural']!='verify':
        continue
    vis+=1
    
    meta_layout=from_meta_to_layout_verify(id_to_prog[name])
    total[name]=meta_layout
print (len(total))

2252


In [41]:
pkl.dump(total,
         open(os.path.join('../mdetr/meta_generated_layout','dev_verify.pkl'),'wb'))

In [95]:
random.shuffle(names)
vis=0
last_verify=defaultdict(int)
op_dict=defaultdict(int)
for k,name in enumerate(names):
    row=gqa_val_q[name]
    if row['types']['structural']!='verify':
        continue
    vis+=1
    
    meta=id_to_prog[name]
    for step in meta:
        info=step.split('(')
        op=info[0].split('_')[0]
        op_dict[op]+=1
        arg=info[1][:-1].split(',')
        if op=='verify':
            last_verify[info[0]]+=1
print(op_dict)
print (last_verify)

defaultdict(<class 'int'>, {'select': 26534, 'relate': 13075, 'verify': 16865, 'filter': 6555, 'exist': 10548})
defaultdict(<class 'int'>, {'verify': 5184, 'verify_rel_inv': 579, 'verify_rel': 7344, 'verify_f': 879, 'verify_v': 897, 'verify_h': 1982})


In [39]:
random.shuffle(names)
vis=0
op_dict=defaultdict(int)
for k,name in enumerate(names):
    row=gqa_val_q[name]
    if row['types']['structural']!='verify':
        continue
    vis+=1
    
    meta=id_to_prog[name]
    for step in meta:
        info=step.split('(')
        op=info[0].split('_')[0]
        op_dict[op]+=1
        arg=info[1][:-1].split(',')
        
print(op_dict)

defaultdict(<class 'int'>, {'select': 26534, 'relate': 13075, 'exist': 10548, 'verify': 16865, 'filter': 6555})


In [42]:
def from_meta_to_layout_logical(meta):
    layout=[]
    for step in meta:
        info=step.split('(')
        op_info=info[0].split('_')
        op=op_info[0]
        arg=info[1][:-1].split(',')
        #print (op,arg)
        if op in ['select','filter']:
            arg=arg[-1]
            if op=='filter' and len(op_info)>1:
                if op_info[1]=='not':
                    arg='not '+arg
            args=[arg]
        elif op=='relate':
            op='relocate'
            if 'inv' in op_info:
                rela_symbol='s'
            else:
                rela_symbol='o'
            args=[arg[1],rela_symbol]
        elif op=='exist':
            args=['?']
        elif op=='verify':
            if len(op_info)==1:
                layout.append({
                    'operation':'filter',
                    'argument':[arg[-1]]
                })
            elif op_info[1] in ['f','v','h']:
                if len(meta)==1:
                    layout.append({
                        'operation':'select',
                        'argument':['scene']
                    })
                layout.append({
                    'operation':'filter',
                    'argument':[arg[-1]]
                })
            elif 'rel' in op_info[1]:
                if 'inv' in op_info:
                    rela_symbol='s'
                else:
                    rela_symbol='o'
                layout.append({
                    'operation':'relocate',
                    'argument':[arg[1],rela_symbol]
                })
                layout.append({
                    'operation':'filter',
                    'argument':[arg[-1]]
                })
            op='exist'
            args=['?']
        elif op in ['and','or']:
            args=[]
        layout.append({
            'operation':op,
            'argument':args
        })
        if op=='relocate' and len(arg)==3:
            layout.append({
                'operation':'filter',
                'argument':[arg[-1]]
            })
    return layout

In [273]:
random.shuffle(names)
vis=0
for k,name in enumerate(names):
    if vis>10:
        break
    row=gqa_val_q[name]
    layout=row['semantic']
    clean_layout=update_program(layout)
    new_prog=mine_clean_all(clean_layout)
    #if len(new_prog)<6:
    #    continue
    if row['types']['structural']!='logical':
        continue
    vis+=1
    
    print (row['question'])
    meta_layout=from_meta_to_layout_logical(id_to_prog[name])
    print (id_to_prog[name])
    for k,step in enumerate(new_prog):
        meta_info=meta_layout[k]
        print('\t',step['operation'],step['argument'],
              '\t',meta_info['operation'],meta_info['argument']
             ) 
        
    print (id_to_prog[name],'\n')

Are there either snowboards or skis in this image?
['select(snowboard)', 'exist([0])', 'select(ski)', 'exist([2])', 'or([1],[3])']
	 select ['skis'] 	 select ['snowboard']
	 exist ['?'] 	 exist ['?']
	 select ['snowboard'] 	 select ['ski']
	 exist ['?'] 	 exist ['?']
	 or [] 	 or []
['select(snowboard)', 'exist([0])', 'select(ski)', 'exist([2])', 'or([1],[3])'] 

Are there either any dogs or cats in the photograph?
['select(cat)', 'exist([0])', 'select(dog)', 'exist([2])', 'or([1],[3])']
	 select ['cat'] 	 select ['cat']
	 exist ['?'] 	 exist ['?']
	 select ['dog'] 	 select ['dog']
	 exist ['?'] 	 exist ['?']
	 or [] 	 or []
['select(cat)', 'exist([0])', 'select(dog)', 'exist([2])', 'or([1],[3])'] 

Are there both a table and a chair in the picture?
['select(table)', 'exist([0])', 'select(chair)', 'exist([2])', 'and([1],[3])']
	 select ['table'] 	 select ['table']
	 exist ['?'] 	 exist ['?']
	 select ['chair'] 	 select ['chair']
	 exist ['?'] 	 exist ['?']
	 and [] 	 and []
['select(ta

In [43]:
random.shuffle(names)
vis=0
total={}
for k,name in enumerate(names):
    row=gqa_val_q[name]
    layout=row['semantic']
    #clean_layout=update_program(layout)
    #new_prog=mine_clean_all(clean_layout)
    #if len(new_prog)<6:
    #    continue
    if row['types']['structural']!='logical':
        continue
    vis+=1
    
    meta_layout=from_meta_to_layout_logical(id_to_prog[name])
    total[name]=meta_layout
print (len(total))

1803


In [44]:
pkl.dump(total,
         open(os.path.join('../mdetr/meta_generated_layout','dev_logical.pkl'),'wb'))

In [40]:
random.shuffle(names)
vis=0
op_dict=defaultdict(int)
for k,name in enumerate(names):
    row=gqa_val_q[name]
    if row['types']['structural']!='logical':
        continue
    vis+=1
    
    meta=id_to_prog[name]
    for step in meta:
        info=step.split('(')
        op=info[0].split('_')[0]
        op_dict[op]+=1
        arg=info[1][:-1].split(',')
        
print(op_dict)

defaultdict(<class 'int'>, {'select': 28781, 'verify': 7162, 'and': 6527, 'exist': 25030, 'relate': 1358, 'filter': 7254, 'or': 9569})


In [10]:
def from_meta_to_layout_common(meta):
    layout=[]
    for step in meta:
        info=step.split('(')
        op_info=info[0].split('_')
        op=op_info[0]
        arg=info[1][:-1].split(',')
        #print (op,arg)
        if op in ['select','filter']:
            arg=arg[-1]
            if op=='filter' and len(op_info)>1:
                if op_info[1]=='not':
                    arg='not '+arg
            args=[arg]
        elif op in ['common']:
            args=[]
        layout.append({
            'operation':op,
            'argument':args
        })
        if op=='relocate' and len(arg)==3:
            layout.append({
                'operation':'filter',
                'argument':[arg[-1]]
            })
    return layout

In [114]:
random.shuffle(names)
vis=0
for k,name in enumerate(names):
    if vis>10:
        break
    row=gqa_val_q[name]
    layout=row['semantic']
    clean_layout=update_program(layout)
    new_prog=mine_clean_all(clean_layout)
    #if len(new_prog)<6:
    #    continue
    if new_prog[-1]['operation']!='common':
        continue
    vis+=1
    
    print (row['question'])
    meta_layout=from_meta_to_layout_common(id_to_prog[name])
    print (id_to_prog[name])
    for k,step in enumerate(new_prog):
        meta_info=meta_layout[k]
        print('\t',step['operation'],step['argument'],
              '\t',meta_info['operation'],meta_info['argument']
             ) 
        
    print (id_to_prog[name],'\n')

What is common to the wall and the owl?
['select(wall)', 'select(owl)', 'common([0],[1])']
	 select ['wall'] 	 select ['wall']
	 select ['owl'] 	 select ['owl']
	 common [] 	 common []
['select(wall)', 'select(owl)', 'common([0],[1])'] 

What do both the outfit and the sock have in common?
['select(outfit)', 'select(sock)', 'common([0],[1])']
	 select ['outfit'] 	 select ['outfit']
	 select ['sock'] 	 select ['sock']
	 common [] 	 common []
['select(outfit)', 'select(sock)', 'common([0],[1])'] 

What do the sky and the fence have in common?
['select(sky)', 'select(fence)', 'common([0],[1])']
	 select ['sky'] 	 select ['sky']
	 select ['fence'] 	 select ['fence']
	 common [] 	 common []
['select(sky)', 'select(fence)', 'common([0],[1])'] 

What is common to the helmet and the snowboard?
['select(helmet)', 'select(snowboard)', 'common([0],[1])']
	 select ['helmet'] 	 select ['helmet']
	 select ['snowboard'] 	 select ['snowboard']
	 common [] 	 common []
['select(helmet)', 'select(snowboa

In [11]:
random.shuffle(names)
vis=0
total={}
for k,name in enumerate(names):
    row=gqa_val_q[name]
    layout=row['semantic']
    clean_layout=update_program(layout)
    new_prog=mine_clean_all(clean_layout)
    #if len(new_prog)<6:
    #    continue
    if new_prog[-1]['operation']!='common':
        continue
    vis+=1
    
    meta_layout=from_meta_to_layout_common(id_to_prog[name])
    total[name]=meta_layout
print (len(total))

96


In [12]:
pkl.dump(total,
         open(os.path.join('../mdetr/meta_generated_layout','dev_common.pkl'),'wb'))

In [41]:
random.shuffle(names)
vis=0
op_dict=defaultdict(int)
for k,name in enumerate(names):
    row=gqa_val_q[name]
    layout=row['semantic']
    clean_layout=update_program(layout)
    new_prog=mine_clean_all(clean_layout)
    if new_prog[-1]['operation']!='common':
        continue
    vis+=1
    
    meta=id_to_prog[name]
    for step in meta:
        info=step.split('(')
        op=info[0].split('_')[0]
        op_dict[op]+=1
        arg=info[1][:-1].split(',')
        
print(op_dict)

defaultdict(<class 'int'>, {'select': 1380, 'common': 690})


In [13]:
def from_meta_to_layout_compare(meta):
    layout=[]
    for step in meta:
        info=step.split('(')
        op_info=info[0].split('_')
        op=op_info[0]
        arg=info[1][:-1].split(',')
        #print (op,arg)
        if op in ['select','filter']:
            arg=arg[-1]
            if op=='filter' and len(op_info)>1:
                if op_info[1]=='not':
                    arg='not '+arg
            args=[arg]
        elif op in ['different','same']:
            #print (op_info)
            if len(arg)<3:
                name='type'
            else:
                name=arg[-1]
            args=[op,name]
            op='compare'
        layout.append({
            'operation':op,
            'argument':args
        })
        if op=='relocate' and len(arg)==3:
            layout.append({
                'operation':'filter',
                'argument':[arg[-1]]
            })
    return layout

In [125]:
random.shuffle(names)
vis=0
for k,name in enumerate(names):
    if vis>10:
        break
    row=gqa_val_q[name]
    layout=row['semantic']
    clean_layout=update_program(layout)
    new_prog=mine_clean_all(clean_layout)
    #if len(new_prog)<6:
    #    continue
    if new_prog[-1]['operation']!='compare':
        continue
    vis+=1
    
    print (row['question'])
    meta_layout=from_meta_to_layout_compare(id_to_prog[name])
    print (id_to_prog[name])
    for k,step in enumerate(new_prog):
        meta_info=meta_layout[k]
        print('\t',step['operation'],step['argument'],
              '\t',meta_info['operation'],meta_info['argument']
             ) 
        
    print (id_to_prog[name],'\n')

Is the bag different in color than the toilet?
['select(bag)', 'select(toilet)', 'different_attr([0],[1],color)']
	 select ['bag'] 	 select ['bag']
	 select ['toilet'] 	 select ['toilet']
	 compare ['different', 'color'] 	 compare ['different', 'color']
['select(bag)', 'select(toilet)', 'different_attr([0],[1],color)'] 

Are both the house and the frisbee the same color?
['select(house)', 'select(frisbee)', 'same_attr([0],[1],color)']
	 select ['frisbee'] 	 select ['house']
	 select ['house'] 	 select ['frisbee']
	 compare ['same', 'color'] 	 compare ['same', 'color']
['select(house)', 'select(frisbee)', 'same_attr([0],[1],color)'] 

Are both the jeans and the trousers the same color?
['select(jean)', 'select(pant)', 'same_attr([0],[1],color)']
	 select ['jeans'] 	 select ['jean']
	 select ['pants'] 	 select ['pant']
	 compare ['same', 'color'] 	 compare ['same', 'color']
['select(jean)', 'select(pant)', 'same_attr([0],[1],color)'] 

Do the frame and the door have a different colors?
[

In [23]:
random.shuffle(names)
vis=0
total={}
for k,name in enumerate(names):
    row=gqa_val_q[name]
    layout=row['semantic']
    clean_layout=update_program(layout)
    new_prog=mine_clean_all(clean_layout)
    #if len(new_prog)<6:
    #    continue
    if new_prog[-1]['operation']!='compare':
        continue
    vis+=1
    
    meta_layout=from_meta_to_layout_compare(id_to_prog[name])
    total[name]=meta_layout
    
print (len(total))

470


In [24]:
pkl.dump(total,
         open(os.path.join('../mdetr/meta_generated_layout','dev_compare.pkl'),'wb'))

In [42]:
random.shuffle(names)
vis=0
op_dict=defaultdict(int)
for k,name in enumerate(names):
    row=gqa_val_q[name]
    layout=row['semantic']
    clean_layout=update_program(layout)
    new_prog=mine_clean_all(clean_layout)
    if new_prog[-1]['operation']!='compare':
        continue
    vis+=1
    
    meta=id_to_prog[name]
    for step in meta:
        info=step.split('(')
        op=info[0].split('_')[0]
        op_dict[op]+=1
        arg=info[1][:-1].split(',')
        
print(op_dict)

defaultdict(<class 'int'>, {'select': 6013, 'different': 1450, 'same': 1791})


In [15]:
def from_meta_to_layout_choose(meta):
    layout=[]
    for step in meta:
        info=step.split('(')
        op_info=info[0].split('_')
        op=op_info[0]
        arg=info[1][:-1].split(',')
        #print (op,arg)
        if op in ['select','filter']:
            arg=arg[-1]
            if op=='filter' and len(op_info)>1:
                if op_info[1]=='not':
                    arg='not '+arg
            args=[arg]
        elif op=='relate':
            op='relocate'
            if 'inv' in op_info:
                rela_symbol='s'
            else:
                rela_symbol='o'
            args=[arg[1],rela_symbol]
        elif op=='choose':
            args=[arg[-2],arg[-1]]
            if 'rel' in op_info:
                layout.append({
                    'operation':'select',
                    'argument':[arg[1]]
                })
            if info[0]=='choose_f' and len(meta)==1:
                layout.append({
                    'operation':'select',
                    'argument':['scene']
                })
        layout.append({
            'operation':op,
            'argument':args
        })
        if op=='relocate' and len(arg)==3:
            layout.append({
                'operation':'filter',
                'argument':[arg[-1]]
            })
    return layout

In [161]:
random.shuffle(names)
vis=0
for k,name in enumerate(names):
    if vis>10:
        break
    row=gqa_val_q[name]
    layout=row['semantic']
    clean_layout=update_program(layout)
    new_prog=mine_clean_all(clean_layout)
    #if len(new_prog)<6:
    #    continue
    if new_prog[-1]['operation']!='choose':
        continue
    vis+=1
    
    print (row['question'])
    meta_layout=from_meta_to_layout_choose(id_to_prog[name])
    print (id_to_prog[name])
    for k,step in enumerate(new_prog):
        meta_info=meta_layout[k]
        print('\t',step['operation'],step['argument'],
              '\t',meta_info['operation'],meta_info['argument']
             ) 
        
    print (id_to_prog[name],'\n')

Is this a sandwich or a donut?
['select(this)', 'choose_n([0],sandwich,donut)']
	 select ['this'] 	 select ['this']
	 choose ['donut', 'sandwich'] 	 choose ['sandwich', 'donut']
['select(this)', 'choose_n([0],sandwich,donut)'] 

Is he to the right or to the left of the cup?
['select(he)', 'choose_rel_inv([0],man,to the right of,to the left of)']
	 select ['he'] 	 select ['he']
	 select ['man'] 	 select ['man']
	 choose ['to the right of', 'to the left of'] 	 choose ['to the right of', 'to the left of']
['select(he)', 'choose_rel_inv([0],man,to the right of,to the left of)'] 

Is the black chair to the left or to the right of the sofa the table is in front of?
['select(table)', 'relate_name([0],in front of,sofa)', 'choose_rel_inv([1],chair,to the right of,to the left of)']
	 select ['table'] 	 select ['table']
	 relocate ['in front of', 'o'] 	 relocate ['in front of', 'o']
	 filter ['sofa'] 	 filter ['sofa']
	 select ['chair'] 	 select ['chair']
	 choose ['to the left of', 'to the right

In [16]:
random.shuffle(names)
vis=0
total={}
for k,name in enumerate(names):

    row=gqa_val_q[name]
    layout=row['semantic']
    clean_layout=update_program(layout)
    new_prog=mine_clean_all(clean_layout)
    #if len(new_prog)<6:
    #    continue
    if new_prog[-1]['operation']!='choose':
        continue
    vis+=1
    
    meta_layout=from_meta_to_layout_choose(id_to_prog[name])
    total[name]=meta_layout
print (len(total))

1152


In [17]:
pkl.dump(total,
         open(os.path.join('../mdetr/meta_generated_layout','dev_choose.pkl'),'wb'))

In [126]:
random.shuffle(names)
vis=0
last_op=defaultdict(int)
op_dict=defaultdict(int)
for k,name in enumerate(names):
    row=gqa_val_q[name]
    layout=row['semantic']
    clean_layout=update_program(layout)
    new_prog=mine_clean_all(clean_layout)
    if new_prog[-1]['operation']!='choose':
        continue
    vis+=1
    
    meta=id_to_prog[name]
    for step in meta:
        info=step.split('(')
        op=info[0].split('_')[0]
        op_dict[op]+=1
        arg=info[1][:-1].split(',')
        if op=='choose':
            last_op[info[0]]+=1
        
print(op_dict)
print (last_op)

defaultdict(<class 'int'>, {'choose': 16455, 'select': 16125, 'filter': 4201, 'relate': 3741})
defaultdict(<class 'int'>, {'choose_f': 556, 'choose_attr': 3485, 'choose_rel_inv': 5327, 'choose_h': 1998, 'choose_v': 2359, 'choose_n': 1858, 'choose': 646, 'choose_subj': 226})


In [43]:
random.shuffle(names)
vis=0
op_dict=defaultdict(int)
for k,name in enumerate(names):
    row=gqa_val_q[name]
    layout=row['semantic']
    clean_layout=update_program(layout)
    new_prog=mine_clean_all(clean_layout)
    if new_prog[-1]['operation']!='choose':
        continue
    vis+=1
    
    meta=id_to_prog[name]
    for step in meta:
        info=step.split('(')
        op=info[0].split('_')[0]
        op_dict[op]+=1
        arg=info[1][:-1].split(',')
        
print(op_dict)

defaultdict(<class 'int'>, {'select': 16125, 'choose': 16455, 'relate': 3741, 'filter': 4201})


In [196]:
#altenative layouts
#not generated completely 
#single && pros
#for logical: order inversed

import inflect
stemmer = inflect.engine()

In [244]:
def acc_analysis(gt_layout,meta_layout):
    #compute the exact match between layouts
    #return two scores: layout acc and arg/layout acc
    arg_flag=True
    find_flag=True
    if len(gt_layout) != len(meta_layout):
        return 0.0,0.0,0.0
    for k,step in enumerate(gt_layout):
        gt_op=step['operation']
        meta_op=meta_layout[k]['operation']
        if gt_op != meta_op:
            return 0.0,0.0,0.0
        if gt_op in ['select','filter']:
            gt_arg=[]
            for arg in step['argument']:
                obj=stemmer.singular_noun(arg.lower())
                if obj==False:
                    obj=arg.lower()
                gt_arg.append(obj)
        else:
            gt_arg=[arg.lower() for arg in step['argument']]
        meta_arg=[arg.lower() for arg in meta_layout[k]['argument']]
        
        if gt_op in ['select','filter','query']:
            if gt_arg[0]!=meta_arg[0]:
                #print(gt_arg[0],meta_arg[0])
                arg_flag=False
                if gt_op in ['select','filter']:
                    find_flag=False
        elif gt_op in ['compare','relocate','choose']:
            for arg in gt_arg:
                if arg not in meta_arg:
                    #print (arg,meta_arg)
                    arg_flag=False
    if arg_flag:
        return 1.0,1.0,find_flag
    else:
        return 1.0,0.0,find_flag

In [19]:
q_types=['query','verify','compare','common','choose','logical']
scenic_needed={q_t:[] for q_t in q_types}

In [246]:
meta_query=load_pkl(os.path.join('../mdetr/meta_generated_layout','query.pkl'))
print (len(meta_query))

68167


In [247]:
random.shuffle(names)
vis=0

layout_acc=0.0
arg_acc=0.0

for k,name in enumerate(names):
    row=gqa_val_q[name]
    layout=row['semantic']
    clean_layout=update_program(layout)
    new_prog=mine_clean_all(clean_layout)
    #if len(new_prog)<6:
    #    continue
    if new_prog[-1]['operation']!='query':
        continue
    vis+=1
    
    
    meta_layout=meta_query[name]
    l_acc,a_acc,find_flag=acc_analysis(new_prog,meta_layout)
    
    """if find_flag==False:
        print ('Find:',id_to_prog[name])
        for k,step in enumerate(new_prog):
            meta_info=meta_layout[k]
            print('\t',step['operation'],step['argument'],
                  '\t',meta_info['operation'],meta_info['argument']
                 ) 
        print ('\n')
    if a_acc==0.0:
        print ('Arg:',id_to_prog[name])
        for k,step in enumerate(new_prog):
            meta_info=meta_layout[k]
            print('\t',step['operation'],step['argument'],
                  '\t',meta_info['operation'],meta_info['argument']
                 ) 
        print ('\n')"""
    layout_acc+=l_acc
    arg_acc+=a_acc
    if find_flag==False:
        scenic_needed['query'].append(name)
print ('Layout acc:',layout_acc*100.0/vis)
print ('Argument acc:',arg_acc*100.0/vis)
print ('Scenic needed:',len(scenic_needed['query']))

Layout acc: 99.71980577112092
Argument acc: 90.30909384306189
Scenic needed: 5399


In [254]:
meta_query=load_pkl(os.path.join('../mdetr/meta_generated_layout','verify.pkl'))
print (len(meta_query))

27413


In [261]:
random.shuffle(names)
vis=0

scenic_needed['verify']=[]
layout_acc=0.0
arg_acc=0.0

for k,name in enumerate(names):
    row=gqa_val_q[name]
    layout=row['semantic']
    clean_layout=update_program(layout)
    new_prog=mine_clean_all(clean_layout)
    #if len(new_prog)<6:
    #    continue
    if row['types']['structural']!='verify':
        continue
    vis+=1
    
    
    meta_layout=meta_query[name]
    l_acc,a_acc,find_flag=acc_analysis(new_prog,meta_layout)
    
    """if find_flag==False:
        print ('Find:',id_to_prog[name])
        for k,step in enumerate(new_prog):
            meta_info=meta_layout[k]
            print('\t',step['operation'],step['argument'],
                  '\t',meta_info['operation'],meta_info['argument']
                 ) 
        print ('\n')
    if a_acc==0.0:
        print ('Arg:',id_to_prog[name])
        for k,step in enumerate(new_prog):
            meta_info=meta_layout[k]
            print('\t',step['operation'],step['argument'],
                  '\t',meta_info['operation'],meta_info['argument']
                 ) 
        print ('\n')"""
    layout_acc+=l_acc
    arg_acc+=a_acc
    if find_flag==False:
        scenic_needed['verify'].append(name)
print ('Layout acc:',layout_acc*100.0/vis)
print ('Argument acc:',arg_acc*100.0/vis)
print ('Scenic needed:',len(scenic_needed['verify']))

Layout acc: 97.98635683799657
Argument acc: 84.15715171633896
Scenic needed: 4330


In [276]:
meta_query=load_pkl(os.path.join('../mdetr/meta_generated_layout','logical.pkl'))
print (len(meta_query))

16096


In [280]:
random.shuffle(names)
vis=0

layout_acc=0.0
arg_acc=0.0

scenic_needed['logical']=[]
for k,name in enumerate(names):
    
    row=gqa_val_q[name]
    layout=row['semantic']
    clean_layout=update_program(layout)
    new_prog=mine_clean_all(clean_layout)
    #if len(new_prog)<6:
    #    continue
    if row['types']['structural']!='logical':
        continue
    vis+=1
    
    
    meta_layout=meta_query[name]
    l_acc,a_acc,find_flag=acc_analysis(new_prog,meta_layout)
    
    """if find_flag==False:
        print ('Find:',id_to_prog[name])
        for k,step in enumerate(new_prog):
            meta_info=meta_layout[k]
            print('\t',step['operation'],step['argument'],
                  '\t',meta_info['operation'],meta_info['argument']
                 ) 
        print ('\n')
    if a_acc==0.0:
        print ('Arg:',id_to_prog[name])
        for k,step in enumerate(new_prog):
            meta_info=meta_layout[k]
            print('\t',step['operation'],step['argument'],
                  '\t',meta_info['operation'],meta_info['argument']
                 ) 
        print ('\n')"""
    layout_acc+=l_acc
    arg_acc+=a_acc
    if find_flag==False:
        scenic_needed['logical'].append(name)
print ('Layout acc:',layout_acc*100.0/vis)
print ('Argument acc:',arg_acc*100.0/vis)
print ('Scenic needed:',len(scenic_needed['logical']))

Layout acc: 99.96893638170974
Argument acc: 64.74900596421472
Scenic needed: 5674


In [281]:
meta_query=load_pkl(os.path.join('../mdetr/meta_generated_layout','common.pkl'))
print (len(meta_query))

690


In [282]:
random.shuffle(names)
vis=0

layout_acc=0.0
arg_acc=0.0

scenic_needed['common']=[]
for k,name in enumerate(names):
    row=gqa_val_q[name]
    layout=row['semantic']
    clean_layout=update_program(layout)
    new_prog=mine_clean_all(clean_layout)
    #if len(new_prog)<6:
    #    continue
    if new_prog[-1]['operation']!='common':
        continue
    vis+=1
    
    
    meta_layout=meta_query[name]
    l_acc,a_acc,find_flag=acc_analysis(new_prog,meta_layout)
    
    """if find_flag==False:
        print ('Find:',id_to_prog[name])
        for k,step in enumerate(new_prog):
            meta_info=meta_layout[k]
            print('\t',step['operation'],step['argument'],
                  '\t',meta_info['operation'],meta_info['argument']
                 ) 
        print ('\n')
    if a_acc==0.0:
        print ('Arg:',id_to_prog[name])
        for k,step in enumerate(new_prog):
            meta_info=meta_layout[k]
            print('\t',step['operation'],step['argument'],
                  '\t',meta_info['operation'],meta_info['argument']
                 ) 
        print ('\n')"""
    layout_acc+=l_acc
    arg_acc+=a_acc
    if find_flag==False:
        scenic_needed['common'].append(name)
print ('Layout acc:',layout_acc*100.0/vis)
print ('Argument acc:',arg_acc*100.0/vis)
print ('Scenic needed:',len(scenic_needed['common']))

Layout acc: 100.0
Argument acc: 82.6086956521739
Scenic needed: 120


In [283]:
meta_query=load_pkl(os.path.join('../mdetr/meta_generated_layout','compare.pkl'))
print (len(meta_query))

3241


In [284]:
random.shuffle(names)
vis=0

layout_acc=0.0
arg_acc=0.0

scenic_needed['compare']=[]
for k,name in enumerate(names):
    row=gqa_val_q[name]
    layout=row['semantic']
    clean_layout=update_program(layout)
    new_prog=mine_clean_all(clean_layout)
    #if len(new_prog)<6:
    #    continue
    if new_prog[-1]['operation']!='compare':
        continue
    vis+=1
    
    
    meta_layout=meta_query[name]
    l_acc,a_acc,find_flag=acc_analysis(new_prog,meta_layout)
    
    """if find_flag==False:
        print ('Find:',id_to_prog[name])
        for k,step in enumerate(new_prog):
            meta_info=meta_layout[k]
            print('\t',step['operation'],step['argument'],
                  '\t',meta_info['operation'],meta_info['argument']
                 ) 
        print ('\n')
    if a_acc==0.0:
        print ('Arg:',id_to_prog[name])
        for k,step in enumerate(new_prog):
            meta_info=meta_layout[k]
            print('\t',step['operation'],step['argument'],
                  '\t',meta_info['operation'],meta_info['argument']
                 ) 
        print ('\n')"""
    layout_acc+=l_acc
    arg_acc+=a_acc
    if find_flag==False:
        scenic_needed['compare'].append(name)
print ('Layout acc:',layout_acc*100.0/vis)
print ('Argument acc:',arg_acc*100.0/vis)
print ('Scenic needed:',len(scenic_needed['compare']))

Layout acc: 100.0
Argument acc: 65.56618327676642
Scenic needed: 1000


In [285]:
meta_query=load_pkl(os.path.join('../mdetr/meta_generated_layout','choose.pkl'))
print (len(meta_query))

16455


In [286]:
random.shuffle(names)
vis=0

layout_acc=0.0
arg_acc=0.0

scenic_needed['choose']=[]
for k,name in enumerate(names):
    row=gqa_val_q[name]
    layout=row['semantic']
    clean_layout=update_program(layout)
    new_prog=mine_clean_all(clean_layout)
    #if len(new_prog)<6:
    #    continue
    if new_prog[-1]['operation']!='choose':
        continue
    vis+=1
    
    
    meta_layout=meta_query[name]
    l_acc,a_acc,find_flag=acc_analysis(new_prog,meta_layout)
    
    """if find_flag==False:
        print ('Find:',id_to_prog[name])
        for k,step in enumerate(new_prog):
            meta_info=meta_layout[k]
            print('\t',step['operation'],step['argument'],
                  '\t',meta_info['operation'],meta_info['argument']
                 ) 
        print ('\n')
    if a_acc==0.0:
        print ('Arg:',id_to_prog[name])
        for k,step in enumerate(new_prog):
            meta_info=meta_layout[k]
            print('\t',step['operation'],step['argument'],
                  '\t',meta_info['operation'],meta_info['argument']
                 ) 
        print ('\n')"""
    layout_acc+=l_acc
    arg_acc+=a_acc
    if find_flag==False:
        scenic_needed['choose'].append(name)
print ('Layout acc:',layout_acc*100.0/vis)
print ('Argument acc:',arg_acc*100.0/vis)
print ('Scenic needed:',len(scenic_needed['choose']))

Layout acc: 99.829838954725
Argument acc: 83.90154968094804
Scenic needed: 2321


In [ ]:
for q_t in q_types:
    print (q_t,len(scenic_needed[q_t]))

In [25]:
scenic_needed={}
for q_t in q_types:
    meta_layout=load_pkl(
        os.path.join('../mdetr/meta_generated_layout','dev_'+q_t+'.pkl')
    )
    #print (q_t,len(meta_layout),len(scenic_needed[q_t]))
    for name in meta_layout:
        scenic_needed[name]=meta_layout[name]
    #scenic_needed.extend(meta_layout)
    print (len(scenic_needed))    
    #for name in scenic_needed[q_t]:
    #    layouts[name]=meta_layout[name]
    #print ('\t',len(layouts))
print (len(scenic_needed))

6805
9057
9527
9623
10775
12578
12578


In [26]:
pkl.dump(scenic_needed,
         open(os.path.join('../mdetr/meta_generated_layout/dev_scenic_needed.pkl'),'wb'))

In [296]:
pkl.dump(scenic_needed,
         open(os.path.join('../mdetr/meta_generated_layout/scenic_needed.pkl'),'wb'))

In [297]:
scenic_names=[]
layouts={}
for q_t in scenic_needed:
    meta_layout=load_pkl(
        os.path.join('../mdetr/meta_generated_layout',q_t+'.pkl')
    )
    print (q_t,len(meta_layout),len(scenic_needed[q_t]))
    scenic_names.extend(scenic_needed[q_t])
    for name in scenic_needed[q_t]:
        layouts[name]=meta_layout[name]
    print ('\t',len(layouts))
print (len(scenic_names))
print (len(layouts))

query 68167 5399
	 5399
verify 27413 4330
	 9729
compare 3241 1000
	 10729
common 690 120
	 10849
choose 16455 2321
	 13170
logical 16096 5674
	 18844
18844
18844


In [298]:
pkl.dump(layouts,
         open(os.path.join('../mdetr/meta_generated_layout/scenic_needed_layout.pkl'),'wb'))

In [28]:
pkl.dump(scenic_needed,
         open(os.path.join('../mdetr/meta_generated_layout/dev_scenic_needed_layout.pkl'),'wb'))

In [29]:
layouts=load_pkl('../mdetr/meta_generated_layout/dev_scenic_needed_layout.pkl')
print(len(layouts))

12578


In [300]:
print(len(names))

132062


In [301]:
for k,name in enumerate(names):
    if name in layouts:
        continue
    row=gqa_val_q[name]
    layout=row['semantic']
    clean_layout=update_program(layout)
    new_prog=mine_clean_all(clean_layout)
    layouts[name]=new_prog

In [302]:
print(len(layouts))

132062


In [303]:
pkl.dump(layouts,
         open(os.path.join('../mdetr/meta_generated_layout/all_meta_layout.pkl'),'wb'))

In [30]:
pkl.dump(scenic_needed,
         open(os.path.join('../mdetr/meta_generated_layout/dev_all_meta_layout.pkl'),'wb'))